For a better vizualisation of this notebook please read using the [notebook viewer](http://nbviewer.jupyter.org/github/cgallay/Ada/blob/master/Homework3/HOMEWORK%203.ipynb)

In [1]:
import pandas as pd
import folium
import json
import os
from pprint import pprint
import branca
from folium import plugins

In [2]:
data = pd.read_csv('data/tipsun20.tsv', sep='\t', na_values=': ')

In [3]:
split = data['sex,age,unit,geo\\time'].str.split(',', 4, expand=True)\
.rename(columns={0:"sex", 1:"age", 2:"unit", 3:"id"})
df = pd.concat([split, data], axis = 1)#.drop('sex,age,unit,geo\\time')
df_total = df.loc[df.age== "TOTAL"]
df_interest = df_total[["id", "2016 "]].rename(columns={'2016 ': 'unemployement'})
df_interest["unemployement"] = df_interest["unemployement"].astype(float)
df_interest.head()

,id,unemployement
0,AT,6.0
1,BE,7.8
2,BG,7.6
3,CY,13.0
4,CZ,4.0


In [4]:
topo_eur_data = json.load(open(r'topojson/europe.topojson.json'))


In [5]:
euro_topo = os.path.join('topojson', 'europe.topojson.json')

m = folium.Map([55.519962, 16.633597], tiles='mapboxbright', zoom_start=4)
plugins.ScrollZoomToggler().add_to(m)
plugins.Fullscreen(
    position='bottomright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True).add_to(m)

serie = df_interest.set_index('id')['unemployement']
colorscale = branca.colormap.linear.YlOrRd.scale(0, serie.max())
colorscale = colorscale.to_step(data=serie, quantiles=[0, 0.1, 0.4, 0.7, 0.85, 0.95, 1])
colorscale.caption = 'Unemployment Rate (%)'
colorscale.add_to(m)

def style_function(feature):
    rate = serie.get(feature['id'], None)
    return {
        'fillOpacity': 0.7,
        'weight': 1,
        'color': 'black',
        'fillColor': '#grey' if rate is None else colorscale(rate)
    }

folium.TopoJson(open(euro_topo),
               'objects.europe',
               style_function=style_function).add_to(m)
m
#ATTENTION CHECK THAT ALL COUNTRY HAVE THE CORRECT TRANSLATION

## Part 2

In [6]:
#load dict for canton to abrevation
dict_trans = pd.read_excel("./data/Canton_map.xlsx").set_index("Canton").to_dict()["Canton_abv"]

In [7]:
swiss_topo = r'topojson/ch-cantons.topojson.json'

def add_swiss_map(m, df: pd.DataFrame, column: str, scale: int):
    serie = df.set_index("Canton_abv")[column]
    def style_function(feature):
        rate = serie.get(feature['id'], None)
        return {
            'fillOpacity': 0.5,
            'weight': 1,
            'fillColor': '#black' if rate is None else colorscale(rate)
        }
    colorscale = branca.colormap.linear.YlOrRd.scale(0, scale)
    folium.TopoJson(open(swiss_topo),
                   'objects.cantons',
                    name=column,
                   style_function=style_function).add_to(m)
    #m.add_child(colorscale)
    return m
def create_swiss_map(df: pd.DataFrame, column : str):
    serie = df.set_index("Canton_abv")[column]
    def style_function(feature):
        rate = serie.get(feature['id'], None)
        return {
            'fillOpacity': 0.5,
            'weight': 1,
            'fillColor': '#black' if rate is None else colorscale(rate)
        }
    colorscale = branca.colormap.linear.YlOrRd.scale(0, serie.max())
    m = folium.Map([46.8734, 8.2200], zoom_start=8)
    folium.TopoJson(open(swiss_topo),
                   'objects.cantons',
                    name="test",
                   style_function=style_function).add_to(m)
    #m.add_child(colorscale)
    #folium.LayerControl().add_to(m)
    """
    m.choropleth(
        geo_data=open(swiss_topo),
        topojson='objects.cantons',
        data=df,
        columns=['Canton_abv', 'Taux de chômage'],
        key_on='id',
        fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
        legend_name='Unemployment Rate (%)',
        style_function=style_function,
        reset=True)
        """
    return m
def create_swiss_map2(df: pd.DataFrame, column: str):
    m = folium.Map([46.8734, 8.2200], zoom_start=8)
    plugins.ScrollZoomToggler().add_to(m)
    m.choropleth(
        geo_data=open(swiss_topo),
        topojson='objects.cantons',
        data=df,
        columns=['Canton_abv', column],
        key_on='id',
        fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
        legend_name='Unemployment Rate (%)',
        reset=True)
    return m

In [8]:
df_swiss = pd.read_excel('./data/Taux de chômage_base.xlsx')
df_swiss["Taux de chômage"] = df_swiss["Taux de chômage"].astype(float)
create_swiss_map2(df_swiss, "Taux de chômage")

In [9]:
df_swiss["Demandeurs d'emploi chômeurs"] = df_swiss["Chômeurs inscrits"] = df_swiss["Demandeurs d'emploi non chômeurs"] 
df_swiss["Taux de chômage sans employe"] = df_swiss["Demandeurs d'emploi chômeurs"] * df_swiss["Taux de chômage"] / df_swiss["Demandeurs d'emploi"]
create_swiss_map2(df_swiss, "Taux de chômage sans employe")

## Part 3

In [10]:
swiss_df = pd.read_excel("data/Taux de chômage Nationalité.xlsx")
index = ["Région linguistique", "Grande région id", "Nationalité"]
swiss_df["Canton_abv"] = swiss_df["Canton"].replace(dict_trans)
swiss_df_nat = swiss_df.set_index("Nationalité")

In [11]:
scale = max(swiss_df_nat.loc["Suisses"]["Taux de chômage"].max(), \
            swiss_df_nat.loc["Etrangers"]["Taux de chômage"].max())
colorscale = branca.colormap.linear.YlOrRd.scale(0, scale)

def add_swiss_map(m, df: pd.DataFrame):
    serie = df.set_index("Canton_abv")["Taux de chômage"]
    def style_function(feature):
        rate = serie.get(feature['id'], None)
        return {
            'fillOpacity': 0.5,
            'weight': 1,
            'fillColor': '#black' if rate is None else colorscale(rate)
        }
    folium.TopoJson(open(swiss_topo),
                   'objects.cantons',
                    name="my name",
                   style_function=style_function).add_to(m)

    return m

m1 = folium.Map([46.8734, 8.2200], zoom_start=7)
m2 = folium.Map([46.8734, 8.2200], zoom_start=7)
plugins.ScrollZoomToggler().add_to(m1)
plugins.ScrollZoomToggler().add_to(m2)

m1 = add_swiss_map(m1, swiss_df_nat.loc["Suisses"])
m2 = add_swiss_map(m2, swiss_df_nat.loc["Etrangers"])

f = branca.element.Figure()
div = f.add_subplot(1, 2, 1, margin=0.005)
m1.add_to(div)
div2 = f.add_subplot(1, 2, 2, margin=0.005)
colorscale.add_to(m2)
m2.add_to(div2)

f.save(os.path.join('results', 'Features.html'))
f

# Bonus

In [12]:
df_swiss["size_pop"] = df_swiss["Demandeurs d\'emploi"] / df_swiss['Taux de chômage']
group = df_swiss.groupby("Région linguistique").sum()[["Demandeurs d'emploi", "size_pop"]]
group["Taux"] = group["Demandeurs d'emploi"] / group["size_pop"]
#df_swiss["agregate_region"] = 
group

,Demandeurs d'emploi,size_pop,Taux
Région linguistique,,,
Suisse alémanique,120214,48681.042536,2.469421
Suisse romande et Tessin,73410,19065.154059,3.850480
Total,193624,64541.333333,3.000000
